In [1]:
# Daniel loads his data / If we want different data we do that here
from load_DER_data import load_dataset
raw_data = load_dataset()

from datasets_utils import datasets_from_data
inp, ev, pdf = datasets_from_data(raw_data,
                                  sw_len_samples=int(7*24*(60/5)),
                                  fc_len_samples=int(1*25*(60/5)),
                                  fc_steps=25,
                                  fc_tiles=33,
                                  target_dims=[0,1,2,3,4],
                                  plot=True,
                                 steps_to_new_sample=15)
import numpy as np
ev = np.expand_dims(ev, axis=-1)
inp_train = inp[:int(0.8*inp.shape[0]),:,:]
inp_test = inp[int(0.8*inp.shape[0]):,:,:]
pdf_train = pdf[:int(0.8*inp.shape[0]),1:,:]
pdf_test = pdf[int(0.8*inp.shape[0]):,1:,:]
ev_train = ev[:int(0.8*inp.shape[0]),1:,:]
ev_test = ev[int(0.8*inp.shape[0]):,1:,:]
ev_teacher_train = ev[:int(0.8*inp.shape[0]),:-1,:]
ev_teacher_test = ev[int(0.8*inp.shape[0]):,:-1,:]

print('The training set has an input data shape of ', 
      inp_train.shape,
      'to expected value targets of ',
     ev_train.shape,
     'or alternatively pdf_targets of ',
     pdf_train.shape)
print('-----------------------------------------------')
print('The testing set has an input data shape of ', 
      inp_test.shape,
      'to expected value targets of ',
     pdf_test.shape,
     'or alternatively pdf_targets of ',
     ev_test.shape)


fetching NWP...
Simulations start at:  2016-01-01 18:00:00
Sample time of Sims is:  0:05:00
Sample time of simulation: 0:05:00
missing  503  samples from  2016-12-31 00:00:00 to  2017-01-01 18:00:00
Setting flags for those datapoints to 1
(649, 12) (503, 13)
missing  1079  samples from  2017-03-23 00:00:00 to  2017-03-26 18:00:00
Setting flags for those datapoints to 1
(649, 12) (1079, 13)
replacing the fauly values with the mean of the array
fetched NWP ... now fetching PV Data
Loading generation data from  2016-01-0117:00:00  to  2017-12-3117:05:00
fetched PV ... now concatenating


c:\users\danielcmaz\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
c:\users\danielcmaz\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


<Figure size 2000x1500 with 17 Axes>

0 percent converted
9 percent converted
19 percent converted
29 percent converted
39 percent converted
49 percent converted
59 percent converted
69 percent converted
79 percent converted
89 percent converted
The training set has an input data shape of  (10704, 2016, 16) to expected value targets of  (10704, 24, 1) or alternatively pdf_targets of  (10704, 24, 33)
-----------------------------------------------
The testing set has an input data shape of  (2677, 2016, 16) to expected value targets of  (2677, 24, 33) or alternatively pdf_targets of  (2677, 24, 1)


In [3]:
# dumb variation of the encoder decoder
# this will be using teacher forcing
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()
# make sure we are working clean

E_D_layers = 2
E_D_units = 256
out_shape = [ev_train.shape[1], ev_train.shape[2]]
in_shape = [inp_train.shape[1], inp_train.shape[2]]


# Build Encoder Layers 
encoder_inputs = tf.keras.layers.Input(shape=(inp_train.shape[1], inp_train.shape[2]))
# ------------------------------------------------------------------------------------------
layers = E_D_layers # remember units and layers
units = E_D_units
encoder = [] # define the encoder layers
for layer in range(E_D_layers):
    encoder.append(tf.keras.layers.LSTM(units,
                           return_sequences=True,
                           return_state=True,
                           recurrent_initializer='glorot_uniform'))
# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------
# build Decoder
decoder_inputs = tf.keras.layers.Input(shape=(out_shape[0], out_shape[1]))
# ------------------------------------------------------------------------------------------
decoder = []
layers = E_D_layers
units = E_D_units
for layer in range(layers):
    decoder.append(tf.keras.layers.LSTM(units,
                           return_sequences=True,
                           return_state=True,
                           recurrent_initializer='glorot_uniform'))
    
decoder_wrap = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=1))
# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------

# Run Encoder
encoder_states = []
encoder_outputs = []
enc_output_layer = None
for layer in range(layers):
    if layer == 0:
        enc_output_layer, state_h, state_c = encoder[layer](encoder_inputs)
        encoder_states.append([state_h, state_c])
        encoder_outputs.append(enc_output_layer)
    else:
        enc_output_layer, state_h, state_c = encoder[layer](enc_output_layer)
        encoder_states.append([state_h, state_c])
        encoder_outputs.append(enc_output_layer)
        
# Run Decoder with Encoder stuffs
dec_states = []
for layer in range(layers): # feed last timesteps outputs as inputs to the decoder and propagate
    if layer == 0:
        output_layer, state_h, state_c = decoder[layer](decoder_inputs, initial_state=encoder_states[layer])
        dec_states.append([state_h, state_c])
    else:
        output_layer, state_h, state_c = decoder[layer](output_layer, initial_state=encoder_states[layer])
        dec_states.append([state_h, state_c])

decoder_output = decoder_wrap(output_layer)



# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_output)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.MeanAbsolutePercentageError(), tf.keras.metrics.MeanSquaredError()]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.summary()
model.fit(x=[inp_train, ev_teacher_train],
          y=ev_train,
          batch_size=68,
          epochs=30,
          validation_split=0.2)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2016, 16)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 48, 1)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 2016, 256),  279552      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 48, 256), (N 264192      input_2[0][0]                    
                                                                 lstm[0][1]                   

KeyboardInterrupt: 

In [8]:
# dumb variation of the encoder decoder
# this will be using teacher forcing
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()
# make sure we are working clean

E_D_layers = 2
E_D_units = 128
out_shape = [ev_train.shape[1], ev_train.shape[2]]
in_shape = [inp_train.shape[1], inp_train.shape[2]]
blend_factor = 0.5

# Build Encoder Layers 
encoder_inputs = tf.keras.layers.Input(shape=(inp_train.shape[1], inp_train.shape[2]))
# ------------------------------------------------------------------------------------------
layers = E_D_layers # remember units and layers
units = E_D_units
encoder = [] # define the encoder layers
for layer in range(E_D_layers):
    encoder.append(tf.keras.layers.LSTM(units,
                           return_sequences=True,
                           return_state=True,
                           recurrent_initializer='glorot_uniform'))
# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------
# build Decoder
decoder_inputs = tf.keras.layers.Input(shape=(out_shape[0], out_shape[1]))
# ------------------------------------------------------------------------------------------
decoder = []
layers = E_D_layers
units = E_D_units
for layer in range(layers):
    decoder.append(tf.keras.layers.LSTM(units,
                           return_sequences=True,
                           return_state=True,
                           recurrent_initializer='glorot_uniform'))
    
decoder_wrap = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=1))
concat_layer = tf.keras.layers.Concatenate(axis=1)
addition_layer = tf.keras.layers.Add()
mult_ = tf.keras.layers.Multiply
# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------

# Run Encoder
encoder_states = []
encoder_outputs = []
enc_output_layer = None
for layer in range(layers):
    if layer == 0:
        enc_output_layer, state_h, state_c = encoder[layer](encoder_inputs)
        encoder_states.append([state_h, state_c])
        encoder_outputs.append(enc_output_layer)
    else:
        enc_output_layer, state_h, state_c = encoder[layer](enc_output_layer)
        encoder_states.append([state_h, state_c])
        encoder_outputs.append(enc_output_layer)
        
# Run Decoder with Encoder stuffs
dec_states = encoder_states
dec_t_minus_1 = tf.keras.layers.Reshape((1,1))(decoder_inputs[:,0,:])
for layer in range(layers): # feed last timesteps outputs as inputs to the decoder and propagate
    if layer == 0:
        output_layer, state_h, state_c = decoder[layer](dec_t_minus_1, initial_state=dec_states[layer])
        dec_states[layer] = [state_h, state_c] # replace the previous decoder states
    else:
        output_layer, state_h, state_c = decoder[layer](output_layer, initial_state=dec_states[layer])
        dec_states[layer] = [state_h, state_c] # replace the previous decoder states
dec_t_minus_1 = decoder_wrap(output_layer)
decoder_output = dec_t_minus_1
print(decoder_output)

for t in range(out_shape[0]-1):
    buffer = tf.keras.layers.Reshape((1,1))(decoder_inputs[:,t+1,:])
    dec_t_minus_1 = (1-blend_factor)*dec_t_minus_1 + blend_factor*buffer
    for layer in range(layers): # feed last timesteps outputs as inputs to the decoder and propagate
        if layer == 0:
            output_layer, state_h, state_c = decoder[layer](dec_t_minus_1, initial_state=dec_states[layer])
            dec_states[layer] = [state_h, state_c] # replace the previous decoder states
        else:
            output_layer, state_h, state_c = decoder[layer](output_layer, initial_state=dec_states[layer])
            dec_states[layer] = [state_h, state_c] # replace the previous decoder states
    dec_t_minus_1 = decoder_wrap(output_layer)
    decoder_output = concat_layer([decoder_output, dec_t_minus_1])
print(decoder_output)



# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_output)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.MeanAbsolutePercentageError(), tf.keras.metrics.MeanSquaredError()]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.summary()
model.fit(x=[inp_train, ev_teacher_train],
          y=ev_train,
          batch_size=128,
          epochs=50,
          validation_split=0.2)

Tensor("time_distributed/Identity:0", shape=(None, 1, 1), dtype=float32)
Tensor("concatenate_22/Identity:0", shape=(None, 24, 1), dtype=float32)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 24, 1)]      0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [None]               0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2016, 16)]   0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1, 1)    

Train on 8563 samples, validate on 2141 samples
Epoch 1/50
8563/8563 [==============================] - 59s 7ms/sample - loss: 0.3336 - mean_absolute_percentage_error: 180.9517 - mean_squared_error: 0.3336 - val_loss: 0.1976 - val_mean_absolute_percentage_error: 347.6372 - val_mean_squared_error: 0.1976
Epoch 2/50
8563/8563 [==============================] - 26s 3ms/sample - loss: 0.1234 - mean_absolute_percentage_error: 135.5536 - mean_squared_error: 0.1234 - val_loss: 0.1434 - val_mean_absolute_percentage_error: 326.2875 - val_mean_squared_error: 0.1434
Epoch 3/50
8563/8563 [==============================] - 26s 3ms/sample - loss: 0.0831 - mean_absolute_percentage_error: 108.4802 - mean_squared_error: 0.0831 - val_loss: 0.1230 - val_mean_absolute_percentage_error: 211.2864 - val_mean_squared_error: 0.1230
Epoch 4/50
8563/8563 [==============================] - 26s 3ms/sample - loss: 0.0693 - mean_absolute_percentage_error: 103.5302 - mean_squared_error: 0.0693 - val_loss: 0.1135 - va

Epoch 33/50
8563/8563 [==============================] - 26s 3ms/sample - loss: 0.0456 - mean_absolute_percentage_error: 88.5758 - mean_squared_error: 0.0456 - val_loss: 0.1302 - val_mean_absolute_percentage_error: 220.2964 - val_mean_squared_error: 0.1302
Epoch 34/50
8563/8563 [==============================] - 26s 3ms/sample - loss: 0.0429 - mean_absolute_percentage_error: 87.9956 - mean_squared_error: 0.0429 - val_loss: 0.1433 - val_mean_absolute_percentage_error: 234.2799 - val_mean_squared_error: 0.1433
Epoch 35/50
8563/8563 [==============================] - 26s 3ms/sample - loss: 0.0427 - mean_absolute_percentage_error: 86.9764 - mean_squared_error: 0.0427 - val_loss: 0.1324 - val_mean_absolute_percentage_error: 241.1935 - val_mean_squared_error: 0.1324
Epoch 36/50
8563/8563 [==============================] - 26s 3ms/sample - loss: 0.0401 - mean_absolute_percentage_error: 92.8319 - mean_squared_error: 0.0401 - val_loss: 0.1453 - val_mean_absolute_percentage_error: 277.9282 - val